In [28]:
from google.colab import files
uploaded_files = files.upload()  # This opens a file picker dialog in Colab

Saving Case Compendium.pdf to Case Compendium (1).pdf


In [29]:
# ⚙️ SYSTEM SETUP: Install required libraries with pinned versions
!pip install -U \
  torch==2.6.0\
  transformers\
  sentence-transformers\
  faiss-cpu\
  langchain\
  langchain-core\
  langchain-community\
  langchain-groq\
  llama-index\
  pypdf\
  groq\
  streamlit\
  -q python-dotenv\
  --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 32.2 MB/s eta 0:00:00


In [30]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

# Load API key from .env
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Load PDF and split into chunks
loader = PyPDFLoader("Case Compendium.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Define embedding model
model_name = "BAAI/bge-small-en"

model_kwargs = {"device": "cpu"}
encode_kwargs = {'normalize_embeddings': False}
embeddings1 = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    show_progress=True
)

# Extract raw text from Document objects
texts_only = [doc.page_content for doc in texts]
print(len(texts_only))
print(embeddings1)



437
client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-small-en' cache_folder=None model_kwargs={'device': 'cpu'} encode_kwargs={'normalize_embeddings': False} query_instruction='Represent this question for searching relevant passages: ' embed_instruction='' show_progress=True


In [31]:
db=FAISS.from_texts(
    texts=texts_only,
    embedding=embeddings1,
)

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

In [32]:
db.save_local("faiss_index")

In [38]:
%%writefile app.py
import os
import streamlit as st
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.schema.runnable import RunnableLambda
model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

def get_embeddings():
    return HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-small-en",
        model_kwargs={"device": "cpu", "trust_remote_code": True},
        encode_kwargs={"normalize_embeddings": False},
    )

def get_faiss_index(embeddings):
    return FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)

# Safe, persistent loading
embeddings1 = get_embeddings()
db1 = get_faiss_index(embeddings1)
retriever = db1.as_retriever(search_type="similarity", search_kwargs={"k": 5})


llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"), model="llama-3.1-8b-instant")

prompt_template = """
You are an intelligent and tricky interviewer from the comany and industry chosen by candidate tasked with interviewing candidates based on provided context for the company and industry chosen by user.
you should ask questions based on context and query only not on some previous memory.
you should ask questions short and concise, one at a time.

you should be critical but flexible, if user replies to your answer which you think is not right u have to tell them but in two line at most and continue your conversation as as interviewer.
you have access to all previous responses given by him and based on given context, you form questions. and you have to continue like an interviewer. when you information
database ends. write thank you and end the conversation.
Use the following context to respond to the user's question. if you dont find company and industry tell user to write correct comany and industry name.


Context:
{context}

Question:
{query}

Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Page state
if "page" not in st.session_state:
  st.session_state.page = "start"
if "chat_history" not in st.session_state:
  st.session_state.chat_history = []

# ----------------- PAGE NAVIGATION --------------------

if st.session_state.page == "start":
  st.title("Mock Case Interview")
  if st.button("Start", key="start_btn_page1"):
    st.session_state.page = "input"
    st.rerun()

elif st.session_state.page == "input":
  st.title("Company & Industry Setup")
  st.session_state.company = st.text_input("Enter company name:")
  st.session_state.industry = st.text_input("Enter industry:")

  if st.button("Submit",key="start_btn_page2"):
    # Optional: validate company/industry before proceeding
    inp=st.session_state.company+" and "+st.session_state.industry
    st.session_state.chat_history.append(("You", inp))
    chat_history_str = "\n".join([f"{str(sender)}: {str(msg)}" for sender, msg in st.session_state.chat_history])
    response = chain.invoke(chat_history_str)
    st.session_state.chat_history.append(("Interviewer", response))
    st.session_state.page = "chat"
    st.rerun()

  if st.button("End Session",key="start_btn_page3"):

    st.session_state.page = "start"
    st.session_state.chat_history = []
    st.rerun()


elif st.session_state.page == "chat":
    st.title("Your Interviewer is Ready")

    # Display full chat history
    for sender, message in st.session_state.chat_history:
        st.markdown(f"**{sender}**: {message}")

    # Define input box with dynamic default value
    if "user_input" not in st.session_state:
        st.session_state.user_input = ""

    user_query = st.text_input("Your response:", key="input_box", value=st.session_state.user_input)

    # Send Button Logic
    if st.button("Send", key="start_btn_page4"):
        if user_query.strip() != "":
            st.session_state.chat_history.append(("You", user_query.strip()))

            chat_history_str = "\n".join(
                [f"{str(sender)}: {str(msg)}" for sender, msg in st.session_state.chat_history]
            )

            response = chain.invoke(chat_history_str)
            st.session_state.chat_history.append(("Interviewer", response))

            # Clear input and rerun
            st.session_state.user_input = ""  # reset stored input
            st.rerun()
        else:
            st.warning("Please enter a response before sending.")

    # End Session Button
    if st.button("End Session", key="start_btn_page5"):
        st.session_state.page = "start"
        st.session_state.chat_history = []
        st.session_state.user_input = ""
        st.rerun()


Overwriting app.py


In [34]:
!pip install -q streamlit pyngrok
!ngrok config add-authtoken 2ywa7L0yjbbbXQdFMZVGImNbOkC_7M5xiAA1sQR82qn5rDZhT

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [35]:
from pyngrok import ngrok
import threading
import os

# Start public URL tunnel
public_url = ngrok.connect(8501)
print("📣 Open your Streamlit app here:", public_url)

# Run Streamlit in the background
def run_app():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run_app)
thread.start()


📣 Open your Streamlit app here: NgrokTunnel: "https://0630761c53fb.ngrok-free.app" -> "http://localhost:8501"
